# Google Drive Tools

The Google Drive integration in AG2 empowers users to seamlessly access and manage their files within the AG2 framework. With this integration, users can easily list, download, and interact with files stored in their Google Drive, providing a smooth and efficient way to handle documents, images, and other data in real time.

## Installation

Before using Google APIs, you need to enable them in your Google Cloud project. You can enable multiple APIs within a single project.

1. Follow the [Setup environment](https://developers.google.com/workspace/drive/api/quickstart/python#set-up-environment) guide from Google Workspace to get started.
2. Once you've completed the setup, you should have the `credentials.json` file downloaded to your local machine.
3. Install AG2 with the `google-api` extra. Since our examples also use `openai`, install it as well:
   
    ```bash
    pip install -U ag2[openai,google-api]
    ```
    > **Note:** If you have been using `autogen` or `pyautogen`, all you need to do is upgrade it using:  
    > ```bash
    > pip install -U autogen[openai,google-api]
    > ```
    > or  
    > ```bash
    > pip install -U pyautogen[openai,google-api]
    > ```
    > as `pyautogen`, `autogen`, and `ag2` are aliases for the same PyPI package.

You're all set! Now you can start using Google Drive with AG2.

## Imports

In [ ]:
from autogen import AssistantAgent, LLMConfig
from autogen.tools import tool
from autogen.tools.experimental.google import GoogleCredentialsLocalProvider, GoogleDriveToolMap

## Google Authentication

This following code is part of the authentication process for integrating with `Google Drive` within the AG2 framework.
- `client_secret_file`: The path to the client secret file, which you can download from the Google Cloud Console. This file contains the necessary credentials to authenticate the user with Google services.
- `token_file`: The path to a file that will store the access token. After the first run of the script, this token will be saved and used for subsequent authentications, avoiding the need for repeated login processes.
- `provider` (`GoogleCredentialsLocalProvider`): The provider is responsible for managing the authentication process.
- `credentials`: This variable stores the credentials object, which will be used by the system to authenticate the user and gain access to their Google Drive resources.

In [ ]:
client_secret_file = "../credentials.json"
token_file = "../my_token.json"

provider = GoogleCredentialsLocalProvider(
    client_secret_file=client_secret_file,
    scopes=GoogleDriveToolMap.recommended_scopes(),
    token_file=token_file,
)

credentials = provider.get_credentials()

## Agent Configuration

In [ ]:
llm_config = LLMConfig.from_json(
    path="OAI_CONFIG_LIST",
).where(model="gpt-4o-mini")

assistant = AssistantAgent(name="assistant", llm_config=llm_config)

## Register Google Drive Tools

The `GoogleDriveToolMap` class provides a set of predefined tools for seamless interaction with Google Drive. These tools include:
- `list_drive_files_and_folders`: Allows the user to list all files and folders in their Google Drive.
- `download_file_from_drive`: Enables downloading files directly from Google Drive to a specified local folder.

Once the tool map is registered with the agent using `register_for_llm`, the agent becomes capable of suggesting these tool calls.

In [ ]:
google_drive_tool_map = GoogleDriveToolMap(
    credentials=credentials,
    download_folder="ag2_drive_downloads",
)

google_drive_tool_map.register_for_llm(assistant)

## Start the Conversation

With the setup complete, you can now use the assistant to list and download files from your Google Drive.

In [ ]:
assistant.run(
    message="""Get me the last 5 files and download all google-apps.document meme types.
Ignore subfolders for now.
Once done, write 'TERMINATE'.""",
    max_turns=5,
    tools=google_drive_tool_map.tools,
    user_input=False,
)

## Custom GoogleDriveToolMap
You can easily customize the predefined `GoogleDriveToolMap` based on your specific requirements. By subclassing GoogleDriveToolMap, you can modify the toolset available to your agent.

For example:
- **Add custom tools**: You can add new tools, like a `create_summary` tool to generate some specific summary.
- **Remove existing tools**: You can remove unwanted tools, such as the `download_file_from_drive` tool, to better suit your use case.

In [ ]:
class MyGoogleDriveToolMap(GoogleDriveToolMap):
    def __init__(
        self,
        *,
        credentials,
        download_folder,
    ):
        super().__init__(credentials=credentials, download_folder=download_folder)

        @tool(description="Create summary of completed downloads")
        def create_summary(summary: str):
            # Your implementation here
            return "Summary created"

        self.set_tool(create_summary)
        self.remove_tool("download_file_from_drive")

In [ ]:
assistant = AssistantAgent(name="assistant", llm_config=llm_config)

google_drive_tool_map = MyGoogleDriveToolMap(
    credentials=credentials,
    download_folder="ag2_drive_downloads",
)

google_drive_tool_map.register_for_llm(assistant)

assistant.run(
    message="List the latest 10 files and write a short summary based on the file names and meme types.",
    max_turns=4,
    tools=google_drive_tool_map.tools,
    user_input=False,
)